In [1]:
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [24]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') # get data from web
len(df) # len of df, there are 3 tables

3

In [25]:
df_pc_toronto = df[0] # 1st df is our necesary df
df_pc_toronto.shape

(180, 3)

Remove rows with 'Borough' equal 'Not assigned'		

In [26]:
df_pc_toronto.drop(df_pc_toronto[df_pc_toronto['Borough'] == 'Not assigned'].index, axis=0, inplace = True)
df_pc_toronto['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [27]:
df_pc_toronto.shape       

(103, 3)

In [32]:
df_pc_toronto[df_pc_toronto['Neighbourhood'] == 'Not assigned'] # There is no Not assigned cells

,Postal Code,Borough,Neighbourhood


In [38]:
print(df_pc_toronto.head())
print('Shape'+)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
